In [1]:
import pandas as pd
import sqlalchemy
import sqlite3
import requests
from bs4 import BeautifulSoup
import re
import json

In [2]:
data = pd.read_csv('../data/processed/glob_data.csv')

In [3]:
def transform_data(data):
    relevant_data = data[['uuid', 'age', 'country_code', 'normalized_job_code']]
    relevant_data = relevant_data[relevant_data['normalized_job_code'].notna()]
    relevant_data['Quantity'] = relevant_data.groupby('normalized_job_code')['normalized_job_code'].transform('count')
    #relevant_data.rename(columns={'age': 'Age', 'country_code': 'Country', 'normalized_job_code': 'Job Title'},
     #                  inplace=True)
    return relevant_data

In [4]:
relevant_data = transform_data(data)
#relevant_data = relevant_data[relevant_data['normalized_job_code'].notna()]
relevant_data.head(1)

,uuid,age,country_code,normalized_job_code,Quantity
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,57,AT,861a9b9151e11362eb3c77ca914172d0,78


In [5]:
def country_name_import(relevant_data):
    scrap_url = 'https://ec.europa.eu/eurostat/statistics-explained/index.php/Glossary:Country_codes'
    html = requests.get(scrap_url).content
    soup = BeautifulSoup(html, 'lxml')
    table = soup.find_all('td')

    # Scrap countries list and cleaning some mess
    codes = [i.text.strip() for i in table]
    pre_codes_clean = [i for i in codes if i != ""]
    codes_clean = [re.sub(r'\b[()]|[()]\b', '', code) for code in pre_codes_clean]
    codes_clean[3] = codes_clean[3].replace('EL', 'GR')
    codes_clean[55] = codes_clean[55].replace('UK', 'GB')

    # List to Dict and replace values into the column
    countries_codes = {codes_clean[i + 1]: codes_clean[i] for i in range(0, len(codes_clean), 2)}
    clean_data = relevant_data
    #clean_data.dropna(inplace=True) 
    clean_data['country_code'].replace(countries_codes, inplace=True)
    
    return clean_data, countries_codes

In [6]:
clean_data, countries_codes = country_name_import(relevant_data)

In [7]:
if 'Spain' in countries_codes.values():
    print('yesa')

yesa


In [8]:
len(clean_data)

5702

In [9]:
#clean_data['Quantity'] = clean_data.apply(lambda row: row.normalized_job_code., axis = 1) 
#clean_data['Quantity3'] = df['Data3'].groupby(df['Date']).transform('count')
#clean_data['Quantity2'] = 1
clean_data['Quantity'] = clean_data.groupby('normalized_job_code')['normalized_job_code'].transform('count') 

In [10]:
clean_data.head(5)

,uuid,age,country_code,normalized_job_code,Quantity
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,57,Austria,861a9b9151e11362eb3c77ca914172d0,78
3,15626d40-db13-0133-ea5c-0a81e8b09a82,45,Austria,049a3f3a2b5f85cb2971ba77ad66e10c,88
4,24954a70-db98-0133-4a64-0a81e8b09a82,41,Austria,f4b2fb1aa40f661488e2782b6d57ad2f,13
5,28583060-dbe2-0133-ff9c-0a81e8b09a82,26,Austria,27af8700f5577cec835acee2cb90a2ff,10
7,dd743bb0-d9a5-0133-53b2-0a81e8b09a82,47,Austria,c1b670eba9ccb65e7c99f7da116d5b9c,72


In [11]:
def finding_job(clean_data, job_title):
    
    for code in clean_data['normalized_job_code']:
           
        api_url = f'http://api.dataatwork.org/v1/jobs/' + code
        response = requests.get(api_url).json()

        if response['title'] == job_title:
            key_uuid = response['uuid']
            break
    
    return {key_uuid: job_title}

In [12]:
#Select_jobs triggers the functions and finds the uuid tied to job selected and returns {uuid: job_title}
def job_data(clean_data):
    select_job = finding_job(clean_data, 'Data Scientist')
    key_uuid = list(select_job.keys())[0]
    
    return select_job, key_uuid

In [13]:
select_job, key_uuid = job_data(clean_data)

In [14]:
def job_filtering(clean_data, normalized_job_code, key_uuid):
    job_filter = clean_data['normalized_job_code'] == key_uuid
    filtered_data = clean_data[job_filter]
    filtered_data['normalized_job_code'].replace(select_job, inplace=True)
    
    return filtered_data

In [15]:
filtered_data = job_filtering(clean_data, 'normalized_job_code', key_uuid)


/home/sergio/miniconda3/envs/lab_env/lib/python3.7/site-packages/pandas/core/generic.py:6586: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [16]:
print(filtered_data[['country_code','normalized_job_code']].head(3))

    country_code normalized_job_code
206      Belgium      Data Scientist
214      Belgium      Data Scientist
249      Belgium      Data Scientist


In [17]:
#filtered_data.to_csv(f'../data/processed/filtered_data.csv', index=False)

In [18]:
def country_filter(dataframe, col, value):
    filter_country = dataframe[col] == value
    filtered_country = dataframe[filter_country]
    return filtered_country

In [19]:
#At this point, you could choose Filtering by country or not. ie Spain. If empty, selects all.
def country_filtering(filtered_data, country, countries_codes):
    print(country)
    if country == 'All_countries':
        return filtered_data
    elif country in countries_codes.values():
        filtered_data = country_filter(filtered_data, 'country_code', country)
        return filtered_data
    else:
        print('Error: country name not included in the database or type error')

In [20]:
country = 'All_countries'
filtered_data = country_filtering(filtered_data, country, countries_codes)

All_countries


In [22]:
filtered_data.head(10)

,uuid,age,country_code,normalized_job_code,Quantity
206,78156750-d949-0133-1a02-0a81e8b09a82,21,Belgium,Data Scientist,67
214,92c27100-d9d0-0133-c4db-0a81e8b09a82,43,Belgium,Data Scientist,67
249,f30223b0-d950-0133-5b67-0a81e8b09a82,50,Belgium,Data Scientist,67
292,5bbb2320-db06-0133-c0ce-0a81e8b09a82,20,Belgium,Data Scientist,67
532,5f1b17e0-d97f-0133-f63f-0a81e8b09a82,38,Czechia,Data Scientist,67
826,9746cbf0-dd28-0133-6b03-0a81e8b09a82,36,Germany,Data Scientist,67
1008,3c437ba0-dd4d-0133-f96e-0a81e8b09a82,51,Germany,Data Scientist,67
1025,50588d40-dd28-0133-6a98-0a81e8b09a82,44,Germany,Data Scientist,67
1219,2d8ccc10-db17-0133-e957-0a81e8b09a82,48,Germany,Data Scientist,67
1279,25865d60-df19-0133-3655-0a81e8b09a82,50,Germany,Data Scientist,67


In [22]:
def percentage_func(filtered_data):
    filtered_data['Percentage'] = filtered_data['Quantity'] / len(clean_data) * 100
    return filtered_data


In [23]:
wa = filtered_data.iloc[0,4]
wa

67

In [30]:
filtered_data['Percentage2'] = filtered_data.iloc[0,4] / len(clean_data) * 100


/home/sergio/miniconda3/envs/lab_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [ ]:
https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas

In [31]:
filtered_data.head(5)

,uuid,age,country_code,normalized_job_code,Quantity,Percentage,Percentage2
206,78156750-d949-0133-1a02-0a81e8b09a82,21,Belgium,Data Scientist,67,1,1
214,92c27100-d9d0-0133-c4db-0a81e8b09a82,43,Belgium,Data Scientist,67,1,1
249,f30223b0-d950-0133-5b67-0a81e8b09a82,50,Belgium,Data Scientist,67,1,1
292,5bbb2320-db06-0133-c0ce-0a81e8b09a82,20,Belgium,Data Scientist,67,1,1
532,5f1b17e0-d97f-0133-f63f-0a81e8b09a82,38,Czechia,Data Scientist,67,1,1


In [27]:
filtered_data['country_code', 'age', 'normalized_job_code', 'Quantity', 'Percentage']

KeyError: ('country_code', 'age', 'normalized_job_code', 'Quantity', 'Percentage')

In [28]:
data_testeo = pd.read_csv('../data/processed/filtered_data.csv')
data_testeo.head()


,uuid,age,country_code,normalized_job_code
0,78156750-d949-0133-1a02-0a81e8b09a82,21,Belgium,Data Scientist
1,92c27100-d9d0-0133-c4db-0a81e8b09a82,43,Belgium,Data Scientist
2,f30223b0-d950-0133-5b67-0a81e8b09a82,50,Belgium,Data Scientist
3,5bbb2320-db06-0133-c0ce-0a81e8b09a82,20,Belgium,Data Scientist
4,5f1b17e0-d97f-0133-f63f-0a81e8b09a82,38,Czechia,Data Scientist


In [29]:
len(data_testeo['country_code']) == 'Spain'

False

In [30]:
def percentage_func(col):
    len(data_testo['country_code']) == Spain
    
    return len(data_testeo['uuid'])


In [31]:
data_testeo.rename(columns={'age': 'Age', 'country_code': 'Country', 'normalized_job_code': 'Job Title'}, inplace=True) 
data_testeo.head()

,uuid,Age,Country,Job Title
0,78156750-d949-0133-1a02-0a81e8b09a82,21,Belgium,Data Scientist
1,92c27100-d9d0-0133-c4db-0a81e8b09a82,43,Belgium,Data Scientist
2,f30223b0-d950-0133-5b67-0a81e8b09a82,50,Belgium,Data Scientist
3,5bbb2320-db06-0133-c0ce-0a81e8b09a82,20,Belgium,Data Scientist
4,5f1b17e0-d97f-0133-f63f-0a81e8b09a82,38,Czechia,Data Scientist


In [25]:
def quantity_func(col):
    return col.value_counts()


In [28]:
def analyze(filtered_data):
    result = filtered_data.groupby(['Country', 'Age'])['Job Title'].agg(quantity_func).reset_index()
    print(result)
    return result

In [29]:
result = analyze(data_testeo) 

           Country  Age  Job Title
0          Belgium   20          1
1          Belgium   21          1
2          Belgium   43          1
3          Belgium   50          1
4          Czechia   38          1
5          Denmark   25          1
6          Finland   44          1
7           France   16          1
8           France   21          1
9           France   39          1
10          France   43          1
11          France   44          2
12          France   48          1
13         Germany   35          1
14         Germany   36          2
15         Germany   44          1
16         Germany   48          1
17         Germany   49          1
18         Germany   50          1
19         Germany   51          1
20          Greece   39          1
21          Greece   48          1
22         Ireland   38          1
23           Italy   18          1
24           Italy   27          1
25           Italy   29          1
26           Italy   30          1
27           Italy  

In [15]:
type(result)

pandas.core.frame.DataFrame